In [172]:
from prediction import toolbox
from os import path as osp
from prediction.toolbox import BaseFeatureExtractor

subjects_filename = osp.join('ang1.json')
subjects_data = toolbox.load_subjects_data_from_json_file(subjects_filename)

successfully loaded 2 subjects from ang1.json


utils.py will contain the bellow functions.

In [174]:
import numpy as np
import re

#ignore_items
def ex_list(a):
    uid_to_ex = ['Main-TimerPanel-95e88243-90a7-4ca7-8f5d-013576917860', \
             'Main-NextButton-f2cdfde7-4d18-46fd-84a6-dd0cd8a4d482',\
            'Main-ProgressPanel-1eacf76f-f01d-48b0-a812-afd58a48efe6',\
             '']
    return a not in uid_to_ex

#search_substring
def word_out(word, name):
    a = False
    if re.search(word, name) == None:
        a = True
    return a

#check_counter_side
def offside(l1, l2):
    return int(l1[-1:]) % 2 != int(l2[-1:]) % 2 and same_shelf(l1, l2)

#get_shelf_num
def shelf_num(l):
    if l and not isinstance(l, str) and l[-1].isdigit():
        return -1
    y = int(l[-1])
    x = int(l[-3:-2])
    n = x+(y//2)*3
    return n

def same_shelf(l1,l2):
    return shelf_num(l1) == shelf_num(l2)

def unique_list(uni_gazed):
    output = []
    for x in uni_gazed:
        if x not in output:
            output.append(x)
    return output 

def get_dfs(subjects_data):
    dfs = AnalogiesRndTimeTo1stSolution().extract(subjects_data)
    return dfs
def get_dfd(subjects_data):    
    dfd = AnalogiesRndTime1stSolutionToDone().extract(subjects_data)
    return dfd
def get_dfc(subjects_data):
    dfc = AnalogiesTCompletionToDone().extract(subjects_data)
    return dfc


In [115]:
#clean
class AnalogiesTCompletionToDone(BaseFeatureExtractor):
    """atf.tCompletionToDone"""
    name = 'analogies_task_completion_to_done'
    column_prefix = 'tCompletionToDone'

    @staticmethod
    def extract(subjects):
        def get_tcompletion_to_done(rnd):
            end_time = rnd['RoundEndTime']

            if not rnd['SelectedObjectsList']:
                return 0

            last_obj_time = rnd['SelectedObjectsList'][-1]['startTime']
            return end_time - last_obj_time

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_tcompletion_to_done, subjects)
        return df
#AnalogiesTCompletionToDone().extract(subjects_data)

In [117]:
class AnalogiesRndTimeTo1stSolution(BaseFeatureExtractor):
    """time untill the 1st solution""" 
    name = 'time_untill_1st_solution'
    column_prefix = 'r_t1stSolution'

    @staticmethod
    def extract(subjects):
        def r_t1stSolution(rnd):
            c, sol = [], []
            try:
                start_time = rnd['RoundStartTime']
                rnd_num = int(rnd['RoundName'])
                obj_t1 = rnd['SelectedObjectsList'][0]['startTime']
                selected_object = rnd['SelectedObjectsList']
                for obj in selected_object:
                    obj_t2 = obj['startTime']
                    obj_container = obj['ContainerName']
                    c.append([rnd_num, obj_container, obj_t2])

                if rnd_num < 3:
                    sol = obj_t1 - start_time
                elif rnd_num > 2 and rnd_num < 5:
                    for c1, c2 in zip(c, c[1:]):
                        if not sol:
                            if offside(c1[1], c2[1]):
                                sol = c2[2] - start_time
                            elif offside(c2[1], c1[1]):
                                sol = c1[2] - start_time
                if not sol:
                    sol = np.NAN

                return sol
            except IndexError:
                pass

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'analogies', 'Rounds', r_t1stSolution, subjects)
        df.fillna(0, inplace=True)
        return df

#AnalogiesRndTimeTo1stSolution().extract(subjects_data)

In [120]:
class AnalogiesRndTime1stSolutionToDone(BaseFeatureExtractor):
    """time of the 1st solution untill done""" 
    name = 'r_t1stSolutionToDone'

    @staticmethod
    def extract(subjects):
        def get_t1stSolutionToDone(rnd):
            c, sol = [], []
            if 'RoundEndTime' not in rnd.keys():
                return np.nan
            end_time = rnd['RoundEndTime']
            rnd_num = int(rnd['RoundName'])
            if 'SelectedObjectsList' not in rnd.keys():
                return np.NAN
            selected_object = rnd['SelectedObjectsList']
            if not selected_object:
                return np.NAN
            if rnd_num < 3:
                sol = end_time - selected_object[0]['startTime']
            elif 2 < rnd_num < 5:
                for obj in selected_object:
                    obj_t = obj['startTime']
                    obj_container = obj['ContainerName']
                    c.append([obj_container, obj_t])
                if len(c) < 2:
                    return np.NAN
                for c1, c2 in zip(c, c[1:]):
                    if offside(c1[0], c2[0]):
                        sol = end_time - c2[1]
                        break
            if not sol:
                sol = np.NAN
            return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'analogies', 'Rounds', get_t1stSolutionToDone, subjects)
        df.fillna(np.NAN, inplace=True)
        return df

#AnalogiesRndTime1stSolutionToDone().extract(subjects_data)

In [122]:
class AnalogiesTDone(BaseFeatureExtractor):
    """atf.tDone"""
    name = 'analogies_task_done'
    column_prefix = 'tDone'

    @staticmethod
    def extract(subjects):
        def get_tdone(rnd):
            return rnd['RoundEndTime'] - rnd['RoundStartTime']

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_tdone, subjects)
        return df
#AnalogiesTDone().extract(subjects_data)

In [124]:
#added from C#
class AnalogiesTCompletion(BaseFeatureExtractor):
    """"""
    name = 'tCompletion'

    @staticmethod
    def extract(subjects):
        def get_tcompletion(rnd):
            start_time = rnd['RoundStartTime']

            if not rnd['SelectedObjectsList']:
                return 0

            last_obj_time = rnd['SelectedObjectsList'][-1]['startTime']
            return last_obj_time - start_time

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_tcompletion, subjects)
        return df
#AnalogiesTCompletion().extract(subjects_data)

In [126]:
#Done
class AnalogiesRnObjSelectedPerShelf(BaseFeatureExtractor):
    """number of obj user selected per shelf"""
    name = 'r_nObjSelectedPerShelf'

    @staticmethod
    def extract(subjects):
        def get_nObjSelectedPerShelf(rnd):
            obj = rnd['SelectedObjectsList']
            ans = rnd['RoundUserAnswer']
            obcount, ancount, c_ans = 0, 0, 0
            if not obj:
                return np.NAN
            else: 
                for ob in obj:
                    obcount += 1
                for an in ans:
                    for sh in an['Shelves']:
                        if (sh['LeftObject']['name'][:3]!='Ref' and sh['RightObject']['name'][:3]!='Ref') or sh['RightObject']!='NAN/NoAnswer':
                            ancount += 1
                        c_ans = ancount

            n_item = round(c_ans/ obcount ,2)
            return n_item

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nObjSelectedPerShelf, subjects)
        return df


#AnalogiesRnObjSelectedPerShelf().extract(subjects_data)

In [179]:
class AnalogiesRndObjGazedPre1stGrab(BaseFeatureExtractor):
    """number of unique objects gazed before grabbed"""
    name = 'unique_num_objects_gazed_before_grabbed'
    column_prefix = 'r_nObjGazedPre1stGrab'

    @staticmethod
    def extract(subjects):
        def get_nObjGazedPre1stGrab(rnd):
            gazed, held_list, s3, c = [], [], [], 0
            rnd_start = rnd.get('RoundStartTime', [])
            rnd_num = int(rnd['RoundName'])
            gazed_obj = rnd.get('ObjectsGazedAtList', [])
            for gaz in gazed_obj:
                gaze_start = gaz['startTime']
                gaze_len = gaz['length']
                gaze_name = gaz['name']
                gaze_uid = gaz['uid']
                if gaze_len > 0.2 and ex_list(gaze_uid):
                    gazed.append([gaze_name, gaze_start, rnd_num, rnd_start])

            held_obj = rnd['ObjectsHeldList']
            for held in held_obj:
                held_start = held['startTime']
                held_name = held['name']
                held_list.append([held_start, held_name, rnd_num])
            
            for h in held_list:
                for g in gazed:
                    if g[3] < g[1] < h[0]:
                        if word_out('Slot*', g[0]) and word_out('Ref*', g[0]):
                            c += 1
                    elif g[1] >= h[0]:
                        s3 = c
                        c = 0
                        break
            if c > 1:
                s3 = c
            if not s3:
                s3 = np.NAN

            return s3

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nObjGazedPre1stGrab, subjects)
        return df

#AnalogiesRndObjGazedPre1stGrab().extract(subjects_data)

In [181]:
class AnalogiesRndGazedObjPre1stGrab(BaseFeatureExtractor):
    """number of Object gazed before grabbed"""  
    name = 'r_nGazedObjPre1stGrab'

    @staticmethod
    def extract(subjects):    
        def get_nGazedObjPre1stGrab(rnd):
            gazed, held_list, all_gazed, unique_sol = [], [], [], []
            if not rnd['RoundStartTime']:
                return np.NAN
            rnd_start = rnd['RoundStartTime']
            rnd_num = int(rnd['RoundName'])
            gazed_obj = rnd.get('ObjectsGazedAtList', [])
            for gaz in gazed_obj:
                gaze_start = gaz['startTime']
                gaze_len = gaz['length']
                gaze_name = gaz['name']
                gaze_uid = gaz['uid']
                if gaze_len > 0.2 and ex_list(gaze_uid):
                    gazed.append([gaze_name, gaze_start, rnd_num, rnd_start])
            held_obj = rnd['ObjectsHeldList']
            for held in held_obj:
                held_start = held['startTime']
                held_name = held['name']
                held_list.append([held_start, held_name, rnd_num])

            a = 0
            for h in held_list:
                for g in gazed:
                    if h[0] > g[1] > g[3]:
                        if word_out('Slot*', g[0]) and word_out('Ref*', g[0]):
                            all_gazed.append(g[0])
                    elif g[1] > h[0]:
                        unique_sol = len(unique_list(all_gazed))
                        a = unique_sol
                        all_gazed.clear()
                        break
            if not unique_sol:
                unique_sol = np.NAN

            return unique_sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nGazedObjPre1stGrab, subjects)
        return df

#AnalogiesRndGazedObjPre1stGrab().extract(subjects_data) 

,0,1,2,3,4,5,6
id,,,,,,,
5C1C52F6B60C77108DE9C20F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42578675,1.0,NaN,NaN,2.0,2.0,8.0,18.0


In [180]:
class AnalogiesRndGazedObjPreSolution(BaseFeatureExtractor):
    """number of Object gazed before solution"""  
    name = 'r_nGazedObjPreSolution'

    @staticmethod
    def extract(subjects):      
        sol = []
        def get_nGazedObjPreSolution(rnd):
            global sol
            gazed, select_list = [], []
            c = 0
            rnd_start = rnd['RoundStartTime']
            rnd_num =int(rnd['RoundName'])
            gazed_obj = rnd.get('ObjectsGazedAtList', [])
            for gaz in gazed_obj:
                gaze_start = gaz['startTime']
                gaze_len = gaz['length']
                gaze_name = gaz['name']
                gaze_uid = gaz['uid']
                if gaze_len > 0.2 and ex_list(gaze_uid) and word_out('Slot*',gaze_name) and word_out('Ref*',gaze_name):
                    gazed.append([gaze_name, gaze_start, rnd_num, rnd_start])

            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                select_start = select['startTime']
                select_name = select['name']
                cont_name = select['ContainerName']
                select_list.append([select_start, select_name, rnd_num, cont_name])       

            a, shelf1, shelf2 = 0, -1, -1
            for s in select_list:
                for g in gazed:
                    if g[1] > g[3] and g[1] < s[0] and s[2] < 5:
                        c += 1
                        a = c
                    elif g[1] > g[3] and g[1] < s[0] and s[2] > 4:
                        for sl1, sl2 in zip(select_list, select_list[1:]):
                            if offside(sl1[3], sl2[3]) and same_shelf(sl1[3], sl2[3]) and g[1] < sl2[0]:
                                if shelf1 == shelf2:
                                    break
                                else:
                                    c += 1
                                    break
                            elif g[1] >= sl2[0]:
                                shelf1 = shelf_num(sl1[3])
                                sol = c
                                a = c
                                break                   
                    elif g[1] >= s[0]:
                        sol = c
                        a = c
                        c = 0
                        break
            if c > 1:
                sol = c
            elif a < 1 and sol:
                sol = 1
            elif not sol:
                sol = np.nan

            return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nGazedObjPreSolution, subjects)
        return df
#AnalogiesRndGazedObjPreSolution().extract(subjects_data) 

,0,1,2,3,4,5,6
id,,,,,,,
5C1C52F6B60C77108DE9C20F,1,1.0,1,1,1,1,1
42578675,3,NaN,1,3,4,2,8


In [182]:
class AnalogiesRndObjGazedPreSolution(BaseFeatureExtractor):
    """unique number of Object gazed before solution"""   
    name = 'r_nGazedObjPreSolution'

    @staticmethod
    def extract(subjects):      
        def get_nObjGazedPreSolution(rnd):
            gazed, select_list, all_gazed, unique_sol = [], [], [], []
            if not rnd['RoundStartTime']:
                return np.NAN
            rnd_start = rnd['RoundStartTime']
            rnd_num = int(rnd['RoundName'])
            gazed_obj = rnd.get('ObjectsGazedAtList', [])
            for gaz in gazed_obj:
                gaze_start = gaz['startTime']
                gaze_len = gaz['length']
                gaze_name = gaz['name']
                gaze_uid = gaz['uid']
                if gaze_len > 0.2 and ex_list(gaze_uid) and \
                        word_out('Slot*', gaze_name) and word_out('Ref*', gaze_name):
                    gazed.append([gaze_name, gaze_start, rnd_num, rnd_start])

            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                select_start = select['startTime']
                select_name = select['name']
                cont_name = select['ContainerName']
                select_list.append([select_start, select_name, rnd_num, cont_name])

            a, shelf1, shelf2 = 0, -1, -1
            for s in select_list:
                for g in gazed:
                    if s[0] > g[1] > g[3] and s[2] < 5:
                        all_gazed.append(g[0])
                        unique_sol = len(unique_list(all_gazed))
                        a = unique_sol
                    elif s[0] > g[1] > g[3] and s[2] > 4:
                        for sl1, sl2 in zip(select_list, select_list[1:]):
                            if offside(sl1[3], sl2[3]) and same_shelf(sl1[3], sl2[3]) and g[1] < sl2[0]:
                                if shelf1 == shelf2:
                                    all_gazed.append(g[0])
                                    break
                            elif g[1] >= sl2[0]:
                                unique_sol = len(unique_list(all_gazed))
                                a = unique_sol
                                break
                            break
                    elif g[1] > s[0]:
                        unique_sol = len(unique_list(all_gazed))
                        a = unique_sol
                        all_gazed.clear()
                        break

            if a < 1 and unique_sol:
                unique_sol = 1
            elif not unique_sol:
                unique_sol = np.nan
            return unique_sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nObjGazedPreSolution, subjects)
        return df
#AnalogiesRndObjGazedPreSolution().extract(subjects_data)    

,0,1,2,3,4,5,6
id,,,,,,,
5C1C52F6B60C77108DE9C20F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42578675,1.0,NaN,NaN,2.0,3.0,1.0,3.0


In [191]:
class AnalogiesRndGazedObjPre1Selection(BaseFeatureExtractor):
    """number of gazed objects gazed before the 1st selection was done"""
    name = 'r_nGazedObjPre1Selection'

    @staticmethod
    def extract(subjects):  
        sol = []
        def get_nGazedObjPre1Selection(rnd):
            global sol
            gazed, select_list = [], []
            c = 0
            rnd_start = rnd['RoundStartTime']
            rnd_num =int(rnd['RoundName'])
            gazed_obj = rnd.get('ObjectsGazedAtList', [])
            for gaz in gazed_obj:
                gaze_start = gaz['startTime']
                gaze_len = gaz['length']
                gaze_name = gaz['name']
                gaze_uid = gaz['uid']
                if gaze_len > 0.2 and ex_list(gaze_uid) and word_out('Slot*',gaze_name) and word_out('Ref*',gaze_name):
                    gazed.append([gaze_name, gaze_start, rnd_num, rnd_start])

            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                select_start = select['startTime']
                select_name = select['name']
                select_uid = select['uid']
                cont_name = select['ContainerName']
                select_list.append([select_start, select_name, rnd_num, cont_name])       
            print(gazed)
            first, a = True, 0
            for s in select_list:
                if first:
                    first = False
                    for g in gazed:
                        if g[1] > g[3] and g[1] < s[0]:
                            c += 1
                            a = c             
                        elif g[1] >= s[0]:
                            sol = c
                            a = c
                            c = 0
                            break
                else:
                    break

            if c > 1:
                sol = c
            elif not sol:
                sol = np.nan

            return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nGazedObjPre1Selection, subjects)
        return df
#AnalogiesRndGazedObjPre1Selection().extract(subjects_data)

In [192]:
class AnalogiesRndObjGazedPre1Selection(BaseFeatureExtractor):
    """unique number of gazed objects gazed before the 1st selection was done"""
    name = 'r_nObjGazedPre1Selection'

    @staticmethod
    def extract(subjects): 
        def get_nObjGazedPre1Selection(rnd):
            gazed, select_list, all_gazed, unique_sol = [], [], [], []
            rnd_start = rnd['RoundStartTime']
            rnd_num = int(rnd['RoundName'])
            gazed_obj = rnd['ObjectsGazedAtList']
            for gaz in gazed_obj:
                gaze_start = gaz['startTime']
                gaze_len = gaz['length']
                gaze_name = gaz['name']
                gaze_uid = gaz['uid']
                if gaze_len > 0.2 and ex_list(gaze_uid) and \
                        word_out('Slot*', gaze_name) and word_out('Ref*', gaze_name):
                    gazed.append([gaze_name, gaze_start, rnd_num, rnd_start])

            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                select_start = select['startTime']
                select_name = select['name']
                cont_name = select['ContainerName']
                select_list.append([select_start, select_name, rnd_num, cont_name])

            a = 0
            for s in select_list:
                for g in gazed:
                    if s[0] > g[1] > g[3]:
                        all_gazed.append(g[0])
                        unique_sol = len(unique_list(all_gazed))
                        a = unique_sol
                    elif g[1] > s[0]:
                        unique_sol = len(unique_list(all_gazed))
                        a = unique_sol
                        all_gazed.clear()
                        break
                break
                
            if a < 1 and unique_sol:
                unique_sol = 1
            elif not unique_sol:
                unique_sol = np.nan
            else:
                all_gazed.clear()
            return unique_sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nObjGazedPre1Selection, subjects)
        return df
#AnalogiesRndObjGazedPre1Selection().extract(subjects_data)    

,0,1,2,3,4,5,6
id,,,,,,,
5C1C52F6B60C77108DE9C20F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42578675,1.0,NaN,NaN,2.0,2.0,1.0,3.0


In [56]:
#Done
class AnalogiesTimeObjDelta(BaseFeatureExtractor):
    """Time delta between 1st and 2nd object selection"""
    name = 'time_obj_delta'
    column_prefix = 'r_tObjDelta'

    @staticmethod
    def extract(subjects): 
        def get_tObjDelta(rnd):
            select_list, rnd_dlta, delta = [], [], 0.0
            rnd_num =int(rnd['RoundName'])
            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                select_start = select['startTime']
                select_name = select['name']
                cont_name = select['ContainerName']
                if rnd_num > 2:
                    select_list.append([select_start, select_name, rnd_num, cont_name])
            for ob1, ob2 in zip(select_list, select_list[1:]):
                if same_shelf(ob1[3],ob2[3]):
                    delta = ob2[0] - ob1[0]
                    rnd_dlta = delta
                    break
            if not rnd_dlta:
                rnd_dlta = np.NAN
            
            return rnd_dlta

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_tObjDelta, subjects)
        return df
AnalogiesTimeObjDelta().extract(subjects_data)   
    

,0,1,2,3,4,5,6
id,,,,,,,
5BB3E29AAB7BC82E25CDCA50,NaN,NaN,NaN,3.11682,2.33475,2.72600,6.25439
5BB3E81AAB7BC82E25CDCA54,NaN,NaN,NaN,2.16814,2.64653,1.93214,2.79251
5BB49FD069806504BDC24365,NaN,NaN,NaN,1.58747,10.17530,2.27900,5.00341
5BBCAFA13EF34C681ABFDE1D,NaN,NaN,NaN,2.17864,6.28979,4.26538,3.60865


In [109]:
#Done
import pandas as pd
class AnalogiesTimeObjDelta(BaseFeatureExtractor):
    """Time delta between 1st and 2nd object selection"""
    name = 'r_tObjDelta'

    @staticmethod
    def extract(subjects):
        def shelf_time_obj_delta(s_num, selected_list):
            s = (None, 0.0)
            res = np.nan
            for sel in selected_list:
                cont_name = sel['ContainerName']
                if s[0] == None and shelf_num(cont_name) == s_num:
                    s = (cont_name, sel['startTime'])
                elif s[0] != None and cont_name != s[0] and shelf_num(cont_name) == s_num:
                    res = sel['startTime'] - s[1]
                    break
            return res

        def get_tobjdelta(rnd):
            rnd_deltas = []
            rnd_num = int(rnd['RoundName'])
            select_obj = rnd['SelectedObjectsList']
            if rnd_num < 5:
                return shelf_time_obj_delta(3, select_obj)
            elif rnd_num == 5:
                for i in range(3):
                    rnd_deltas.append(shelf_time_obj_delta(i, select_obj))
                return rnd_deltas
            elif rnd_num == 6:
                for i in range(9):
                    rnd_deltas.append(shelf_time_obj_delta(i, select_obj))
                return rnd_deltas

            return np.nan

        res_df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_tobjdelta, subjects)
        df = res_df[res_df.columns[0:5]]
        tmp_df = pd.DataFrame(index=res_df.index)
        for i in range(3):
            tmp_df = tmp_df.join(res_df[5].apply(lambda x: x[i]), rsuffix=str(str(i)))
        for i in range(9):
            tmp_df = tmp_df.join(res_df[6].apply(lambda x: x[i]), rsuffix=str(str(i)))
        tmp_df.columns = range(5, 17)
        df = df.join(tmp_df)
        return df

AnalogiesTimeObjDelta().extract(subjects_data)   
    

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
id,,,,,,,,,,,,,,,,,
5BB3E29AAB7BC82E25CDCA50,NaN,NaN,NaN,3.11682,2.33475,2.72600,3.31060,5.12744,7.80400,3.23900,NaN,6.25439,3.40734,NaN,1.85457,5.80811,NaN
5BB3E81AAB7BC82E25CDCA54,NaN,NaN,NaN,2.16814,5.56060,1.93214,1.96582,4.78051,2.79251,2.90283,2.57984,3.51817,4.92835,2.98316,1.47462,NaN,NaN
5BB49FD069806504BDC24365,NaN,NaN,NaN,1.58747,10.17530,2.27900,2.70256,2.30068,5.00341,1.95459,1.78700,2.56884,2.06617,2.70174,40.59863,5.11377,2.48970
5BBCAFA13EF34C681ABFDE1D,NaN,NaN,NaN,2.17864,6.28979,4.26538,3.22851,2.16838,3.60865,2.88183,4.16602,2.88186,4.79932,NaN,NaN,3.86462,5.10513


In [ ]:
#Done
class AnalogiesRndNumberObjectGrabbed(BaseFeatureExtractor):
    """Number of object grabbed per round"""
    name = 'object_grabbed_per_round'
    column_prefix = 'r_nObjectGrab'

    @staticmethod
    def extract(subjects): 
        def get_nObjectheld(rnd):
            heldc, objects = 0, []
            
            held_obj = rnd['ObjectsHeldList']
            for held in held_obj:
                objects.append(held['name'])           
            heldc = len(unique_list(objects))
            return heldc
        
        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nObjectheld, subjects)
        return df
AnalogiesRndNumberObjectGrabbed().extract(subjects_data)  

In [ ]:
#Done
class AnalogiesRndNumberObjectSelected(BaseFeatureExtractor):
    """Number of object selected per round"""
    name = 'object_selected_per_round'
    column_prefix = 'r_nObjectSelect'

    @staticmethod
    def extract(subjects): 
        def get_nObjectSelect(rnd):
            selectc, objects = 0, []
            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                objects.append(select['name'])
            selectc = len(unique_list(objects))
            return selectc
        
        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nObjectSelect, subjects)
        return df
AnalogiesRndNumberObjectSelected().extract(subjects_data)  

In [ ]:
#Done
class AnalogiesRndNumberSolutions(BaseFeatureExtractor):
    """Number of solution selected per round"""
    name = 'object_selected_per_round'
    column_prefix = 'r_nSolutions'

    @staticmethod
    def extract(subjects): 
        def get_nSolutions(rnd):
            selectc, objects = 0, []
            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                objects.append([select['name'],select['ContainerName']])
            for i1, i2 in zip(objects, objects[1:]):
                if offside(i1[1], i2[1]):
                    selectc += 1
                pass
                    
            return selectc
        
        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nSolutions, subjects)
        return df
AnalogiesRndNumberSolutions().extract(subjects_data)  

In [ ]:
#Done
class AnalogiesRndNumberSelections(BaseFeatureExtractor):
    """Number of object selected per round"""
    name = 'object_selected_per_round'
    column_prefix = 'r_nSelections'

    @staticmethod
    def extract(subjects): 
        def get_nSelections(rnd):
            selectc, objects = 0, []
            select_obj = rnd['SelectedObjectsList']
            for select in select_obj:
                objects.append(select['name'])
            selectc = len(objects)
            return selectc
        
        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nSelections, subjects)
        return df
AnalogiesRndNumberSelections().extract(subjects_data)  

In [ ]:
#Done
class AnalogiesRndNumberSkipped(BaseFeatureExtractor):
    """Number of rounds skipped- needs to be accumulated to a feature"""
    name = 'rounds_skipped'
    column_prefix = 'r_nSkipped'

    @staticmethod
    def extract(subjects):
        def get_nSkipped(rnd):
            skipped, rnd_ans = 0, []
            select_obj = rnd['SelectedObjectsList']
            done = rnd['DoneClicked']
            usr_ans = rnd['RoundUserAnswer']
            for shelf in usr_ans:
                for ans in shelf['Shelves']:
                    ln = ans['LeftObject']['name']
                    rn = ans['RightObject']['name']
                    rnd_ans.append(ln)
                    rnd_ans.append(rn)
            if done and 'NAN/NoAnswer' in rnd_ans:
                print(rnd_ans)
                skipped = 1
            return skipped
        
        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_nSkipped, subjects_data)
        return df
AnalogiesRndNumberSkipped().extract(subjects_data)  

In [ ]:
#Done
class AnalogiesRndNumberGazeToClock(BaseFeatureExtractor):
    """number of times gazed the clock during each round"""
    name = 'gaze_to_clock'
    column_prefix = 'r_nGazeToClock'
    @staticmethod
    def extract(subjects):
        def get_r_nGazeToClock(rnd):
            gaze_count = 0
            gazed_list = rnd['ObjectsGazedAtList']
            for gaze in gazed_list:
                gaze_name = gaze['name']
                if gaze_name.find('Timer') > -1:
                    gaze_count += 1
            return gaze_count
            gaze_count.append()   
                     
        df = toolbox.create_subjects_data_items_multi_dim_df(
            'analogies', 'Rounds', get_r_nGazeToClock, subjects_data)
        return df
AnalogiesRndNumberGazeToClock().extract(subjects_data)  

In [ ]:
class AnalogiesRoundFPostVsPre1stSolutionDur(BaseFeatureExtractor):
    """t1stSolutionToDone/t1stSolution"""
    name = 'r_fPostVsPreCompletionDur'
    column_prefix = 'r_fPostVsPreCompletionDur' 
    @staticmethod
    def extract(subjects_data):
        def get_r_fPostVsPre1stSolutionDur(subjects_data):
            dfs = get_dfs(subjects_data)
            dfd = get_dfd(subjects_data)
            df = dfs / dfd
            return df
        df = get_r_fPostVsPre1stSolutionDur(subjects_data)
        df = df.replace([np.inf, -np.inf], np.nan)
        return df
AnalogiesRoundFPostVsPre1stSolutionDur().extract(subjects_data)

In [ ]:
class AnalogiesRoundFPostVsPreCompletionDur(BaseFeatureExtractor):
    """tCompletionToDone/t1stSolution"""
    name = 'r_fPostVsPreCompletionDur'
    column_prefix = 'r_fPostVsPreCompletionDur'
    @staticmethod
    def extract(subjects_data):
        def get_r_fPostVsPreCompletionDur(subjects_data):
            ndf1 = get_dfs(subjects_data)
            ndf2 = get_dfc(subjects_data)
            #print(ndf1, ndf2)
            ndf = ndf2 / ndf1

            return ndf
        df = get_r_fPostVsPreCompletionDur(subjects_data)
        df = df.replace([np.inf, -np.inf], np.nan)
        return df
df = AnalogiesRoundFPostVsPreCompletionDur().extract(subjects_data)
df


In [ ]:
#Done
class AnalogiesTGazeIrrelevant(BaseFeatureExtractor):
    """time gazing at irrelevant things (i.e. walls, windows etc.), or eyes closed"""
    name = 'time_gaze_solution'
    column_prefix = 'r_tGazeIrrelevant'
    @staticmethod
    def extract(subjects):
        def get_TGazeIrrelevant(rnd):
            irr_tot = 0.0
            irrelevant_items = ['Plane Top Shadow', 'Tiles', 'Logo']
            gaze_list = rnd['ObjectsGazedAtList']
            for g in gaze_list:
                gaze_name = g['name']
                gaze_len = g['length']
                if gaze_name in irrelevant_items:
                    irr_tot += gaze_len
                                   
            return irr_tot

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'analogies', 'Rounds', get_TGazeIrrelevant, subjects)
        return df.sum(axis=1).to_frame()

AnalogiesTGazeIrrelevant().extract(subjects_data)